# KAIST AI605 Assignment 3: Transformer


## Environment
You will only use Python 3.7 and PyTorch 1.9, which is already available on Colab:

In [ ]:
from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.10
torch 1.8.1+cu101


## 1. Attention Layer

We will first start with going over a few concepts that you learned in your high school statistics class.
The variance of a random variable $X$, $\text{Var}(X)$ is defined as $\text{E}[(X-\mu)^2]$ where $\mu$ is the mean of $X$.
Furthermore, given two independent random variables $X$ and $Y$ and a constant $a$,
$$ \text{Var}(X+Y) = \text{Var}(X) + \text{Var}(Y),$$
$$ \text{Var}(aX) = a^2\text{Var}(X),$$
$$ \text{Var}(XY) = \text{E}[X^2]\,\text{E}[Y^2] - (\text{E}[X])^2(\text{E}[Y])^2.$$

> **Problem 1.1** *(3 points)*
  Suppose we are given two sets of $n$ random variables, $X_1 \dots X_n$ and $Y_1 \dots Y_n$,
  where all of these $2n$ variables are mutually independent and have a mean of $0$ and a variance of $1$.
  Prove that
  $$\text{Var}\left(\sum_i^n X_i Y_i\right) = n.$$

> **Solution 1.1**
  Given that $X_i$ and $Y_i$ are mutually independent and have a mean of 0 and a variance of 1, we can write
  $$\text{E}[X_i] = \text{E}[Y_i] = 0$$
  and
  $$\text{Var}(X_i) = \text{E}[(X_i - \mu_i)^2] = \text{E}[X_i^2] = 1,$$
  $$\text{Var}(Y_i) = \text{E}[(Y_i - \mu_i)^2] = \text{E}[Y_i^2] = 1,$$
  for all $i = 1, \dots, n$ where $\mu_i$ is the mean of $X_i$ and $Y_i$ respectively.
  Therefore,
  $$
    \begin{align*}
      \text{Var}\left(\sum_i^n X_i Y_i\right)
      &= \sum_i^n \text{Var}(X_iY_i) \\
      &= \sum_i^n \left(\text{E}[X_i^2]\,\text{E}[Y_i^2] - (\text{E}[X_i])^2(\text{E}[Y_i])^2\right) \\
      &= \sum_i^n (1 \cdot 1 - 0^2 \cdot 0^2) \\
      &= n.
    \end{align*}
  $$

In Lecture 11 and 12, we discussed how the attention is computed in Transformer via the following equation,
  $$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.$$

> **Problem 1.2** *(3 points)* 
  Suppose $Q$ and $K$ are matrices of independent variables each of which has a mean of $0$ and a variance of $1$.
  Using what you learned from Problem 1.1., show that
  $$\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) = 1.$$

> **Solution 1.2**
  Let the matrix $Q \in \mathbb{R}^{n \times d_k}$ and the matrix $K \in \mathbb{R}^{m \times d_m}$.
  The $Q_i$ and the $K_j$ are the row vectors of $Q$ and $K$ respectively, and each of them contains $d_k$ random variables.
  Total $2d_k$ random variables in $Q_i$ and $K_j$ are mutually independent and have a mean of $0$ and a variance of $1$.
  Then, using the result of Problem 1.1, we can rewrite the equation as
  $$
    \begin{align*}
      \text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right)
      &= \frac{1}{d_k} \text{Var}(QK^\top) \\
      &= \frac{1}{d_k} \text{Var}\left(\sum_k^{d_k} Q_{ik} K_{jk}\right) \\
      &= \frac{1}{d_k}
        \begin{bmatrix}
          d_k    & \cdots & d_k \\
          \vdots & \ddots & \vdots \\
          d_k    & \cdots & d_k
        \end{bmatrix} \\
      &= \begin{bmatrix}
          1      & \cdots & 1 \\
          \vdots & \ddots & \vdots \\
          1      & \cdots & 1
        \end{bmatrix} \\
      &= 1.
    \end{align*}
  $$

> **Problem 1.3** *(4 points)*
  What would happen if the assumption that the variance of $Q$ and $K$ is $1$ does not hold?
  Consider each case of it being higher and lower than $1$ and conjecture what it implies, respectively.

> **Solution 1.3**
  If the variance of $Q$ and $K$ is higher than $1$,
  √d_k로 scaling을 해주는 이유는 dot-products의 값이 커질수록 softmax 함수에서 기울기의 변화가 거의 없는 부분으로 가기 때문입니다.
  We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients
  <h2 style="color:red">TODO</p>

## 2. Transformer

In this section, you will implement Transformer for a few tasks that are simpler than machine translation.
First, go through [Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html) and make sure you understand every block of the code.
Then, you will reuse these code where appropriate to create models for following three tasks.
Note that we do not provide a separate training or evaluation data, so it is your job to be able to create these in a reasonable manner.

> **Problem 2.1** *(4 points)*
  Create a model that takes a random set of input symbols from a vocabulary of digits (i.e. 0, 1, ... , 8, 9) as the input and generate back the same symbols.
  Instead of varying length, we fix the length to 32.
  Make sure to report that your model's accuracy (gives credit only if the entire output sequence is correct) goes above 90%.
  Note that a similar problem is also in Annotated Transformer, and copying code is allowed.


> **Problem 2.2** *(6 points)*
  Now, we will implement a bit more useful function, so-called spelling error correction.
  Your job is to create a model whose input is a word with spelling errors, and the output is the spelling-corrected word.
  Here, your vocabulary will be character instead of word.
  You can create your own training data by using an existing text corpus as the target and inject noise into it to use it as the input.
  You are free to use whichever text corpus you like.
  If you can't think of one, please use context data in SQuAD Dataset (see Assignment 2).
  Report accuracy in your own evaluation data (you will receive full credit as long as both the evaluation data and the accuracy are reasonable),
  and also show 5 examples where it succeeds at correcting spelling.

> **Problem 2.3 (bonus)** *(3 points)*
  Extend this word-level spelling correction model to sentence-level.
  You do not have to report accuracy, but find 3 examples where the word-level model fails and sentence-level model correctly predicts.

## Test Model

In [6]:
import copy
import math

import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

In [3]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [4]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [5]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = nn.LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [ ]:
tmp_model = make_model(10, 10, 2)